In [1]:
import os
from typing import overload
import numpy as np
from pymbar import MBAR
import pandas as pd

In [3]:
os.chdir('complex_1_20_200_200/post_processing')

In [4]:
#beta = 1.0 / (300 * 1.380649 * 6.02214076 / 1000)
beta = 1.0 / (300 * 1.380649e-23 )
name_list = [ 0.0, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.35, 0.5, 0.75, 1.0]
#name_list = [ 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.85, 1]

# number of states
K = len(name_list)

mian_dir = os.getcwd()

In [5]:
os.chdir(mian_dir + '/' + str(name_list[0]))
result_path = os.getcwd() + '/MBAR_energy.txt'

In [6]:
u_kn = np.loadtxt(result_path)

In [7]:
u_kn.shape

(12000,)

In [8]:
for file in range(1,len(name_list)):

    os.chdir(mian_dir + '/' + str(name_list[file]))
    result_path = os.getcwd() + '/MBAR_energy.txt'

    a = np.loadtxt(result_path)
    #print(a.shape)
    #print(a)
    u_kn = np.vstack((u_kn,a))

In [9]:
u_kn.shape

(12, 12000)

In [10]:
N_k = np.full(K,u_kn.shape[1]/K, dtype=np.int32)
N_k

array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
       1000], dtype=int32)

In [11]:
mbar = MBAR(u_kn,N_k)

In [12]:
Delta_f_ij, dDelta_f_ij, Theta_ij = mbar.getFreeEnergyDifferences(return_theta=True)

# kbT  -->   kcal/mol   (  *  0.596  )

In [13]:
dDelta_f_ij

array([[0.        , 0.00871004, 0.01649676, 0.02359693, 0.02759416,
        0.03021215, 0.03364135, 0.03601275, 0.04081088, 0.04399281,
        0.04761932, 0.05031266],
       [0.00871004, 0.        , 0.00840246, 0.01660298, 0.02135896,
        0.02448436, 0.02853863, 0.03129116, 0.03671495, 0.0402239 ,
        0.04416155, 0.04705328],
       [0.01649676, 0.00840246, 0.        , 0.00891678, 0.01440655,
        0.01811448, 0.02297159, 0.02624096, 0.03250041, 0.03642065,
        0.04072959, 0.04384863],
       [0.02359693, 0.01660298, 0.00891678, 0.        , 0.00598499,
        0.01026226, 0.01610311, 0.02009783, 0.02759141, 0.03210513,
        0.03692345, 0.04033935],
       [0.02759416, 0.02135896, 0.01440655, 0.00598499, 0.        ,
        0.00448376, 0.01088817, 0.01540348, 0.02390947, 0.02893867,
        0.03419601, 0.03785983],
       [0.03021215, 0.02448436, 0.01811448, 0.01026226, 0.00448376,
        0.        , 0.00666038, 0.01151156, 0.0208103 , 0.02629307,
        0.03195262,

In [14]:
print('Delta_f_ij.shape:',Delta_f_ij.shape)
print(f"Restraint energy : {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol")
with open(f'{mian_dir}/Restraint_energy.txt','w') as f:
    f.write(f'Restraint energy(MBAR): {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol\n')
    for i in range(K-1):
        f.writelines(f'{Delta_f_ij[i][i+1] * 0.596:.3f}\n')

Delta_f_ij.shape: (12, 12)
Restraint energy : 6.001 ± 0.030 Kcal/mol


In [15]:
pd.DataFrame(Delta_f_ij * 0.596)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000000,0.473409,1.010917,1.639885,2.090720,2.443600,2.984032,3.395592,4.251598,4.827636,5.506629,6.002318
1,-0.473409,0.000000,0.537509,1.166477,1.617311,1.970191,2.510623,2.922183,3.778189,4.354227,5.033220,5.528909
2,-1.010917,-0.537509,0.000000,0.628968,1.079802,1.432683,1.973115,2.384674,3.240681,3.816718,4.495712,4.991401
3,-1.639885,-1.166477,-0.628968,0.000000,0.450834,0.803715,1.344147,1.755707,2.611713,3.187750,3.866744,4.362433
4,-2.090720,-1.617311,-1.079802,-0.450834,0.000000,0.352881,0.893313,1.304872,2.160879,2.736916,3.415910,3.911598
5,-2.443600,-1.970191,-1.432683,-0.803715,-0.352881,0.000000,0.540432,0.951992,1.807998,2.384035,3.063029,3.558718
6,-2.984032,-2.510623,-1.973115,-1.344147,-0.893313,-0.540432,0.000000,0.411560,1.267566,1.843603,2.522597,3.018286
7,-3.395592,-2.922183,-2.384674,-1.755707,-1.304872,-0.951992,-0.411560,0.000000,0.856006,1.432044,2.111037,2.606726
8,-4.251598,-3.778189,-3.240681,-2.611713,-2.160879,-1.807998,-1.267566,-0.856006,0.000000,0.576038,1.255031,1.750720
9,-4.827636,-4.354227,-3.816718,-3.187750,-2.736916,-2.384035,-1.843603,-1.432044,-0.576038,0.000000,0.678994,1.174682


In [16]:
O_ij = mbar.computeOverlap()['matrix']
np.savetxt('../restraint_overlap.txt',O_ij,fmt='%.8f')
O_ij.round(decimals=3, out=O_ij)
pd.DataFrame(O_ij)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.345,0.259,0.171,0.093,0.056,0.037,0.019,0.012,0.004,0.002,0.001,0.000
1,0.259,0.230,0.183,0.120,0.082,0.058,0.033,0.021,0.008,0.004,0.002,0.001
2,0.171,0.183,0.177,0.144,0.113,0.088,0.057,0.039,0.016,0.008,0.004,0.002
3,0.093,0.120,0.144,0.152,0.140,0.123,0.093,0.070,0.033,0.018,0.009,0.005
4,0.056,0.082,0.113,0.140,0.145,0.140,0.119,0.097,0.053,0.031,0.016,0.009
5,0.037,0.058,0.088,0.123,0.140,0.144,0.135,0.118,0.072,0.046,0.024,0.015
6,0.019,0.033,0.057,0.093,0.119,0.135,0.146,0.142,0.107,0.075,0.045,0.030
7,0.012,0.021,0.039,0.070,0.097,0.118,0.142,0.150,0.132,0.103,0.068,0.048
8,0.004,0.008,0.016,0.033,0.053,0.072,0.107,0.132,0.165,0.161,0.136,0.113
9,0.002,0.004,0.008,0.018,0.031,0.046,0.075,0.103,0.161,0.185,0.189,0.177


In [17]:
dDelta_f_ij

array([[0.        , 0.00871004, 0.01649676, 0.02359693, 0.02759416,
        0.03021215, 0.03364135, 0.03601275, 0.04081088, 0.04399281,
        0.04761932, 0.05031266],
       [0.00871004, 0.        , 0.00840246, 0.01660298, 0.02135896,
        0.02448436, 0.02853863, 0.03129116, 0.03671495, 0.0402239 ,
        0.04416155, 0.04705328],
       [0.01649676, 0.00840246, 0.        , 0.00891678, 0.01440655,
        0.01811448, 0.02297159, 0.02624096, 0.03250041, 0.03642065,
        0.04072959, 0.04384863],
       [0.02359693, 0.01660298, 0.00891678, 0.        , 0.00598499,
        0.01026226, 0.01610311, 0.02009783, 0.02759141, 0.03210513,
        0.03692345, 0.04033935],
       [0.02759416, 0.02135896, 0.01440655, 0.00598499, 0.        ,
        0.00448376, 0.01088817, 0.01540348, 0.02390947, 0.02893867,
        0.03419601, 0.03785983],
       [0.03021215, 0.02448436, 0.01811448, 0.01026226, 0.00448376,
        0.        , 0.00666038, 0.01151156, 0.0208103 , 0.02629307,
        0.03195262,